In [ ]:
# nse scratchpad initiation
from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

ib =  get_connected('nse', 'live')

In [ ]:
%%time
dynamic(ib)

In [ ]:
# nse dynamic changes

#... get the open orders

open_trades = ib.openTrades()

if open_trades:
    df_open=util.df(open_trades)

    # open contracts
    df_oc = util.df(list(df_open['contract']))
    df_oc = df_oc[list(df_oc)[:6]]
    df_oc = df_oc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # orders
    df_ord = pd.DataFrame([(row.orderId, row.permId, row.action, row.totalQuantity, row.lmtPrice) 
              for idx, row in df_open.order.items()],
            columns = ['orderId', 'permId', 'action', 'totalQuantity', 'lmtPrice'])

    df_openord = df_oc.join(df_ord)

else:
    df_openord = pd.DataFrame() # Empty DataFrame

#.... check for overall position risk. Abort after cancelling SELL orders at risk
df_ac = util.df(ib.accountSummary())
net_liq = float(df_ac[df_ac.tag == 'NetLiquidation'][:1].value)
init_margin = float(df_ac[df_ac.tag == 'InitMarginReq'][:1].value)

if init_margin >= net_liq*ovallmarginlmt:  # if overall limit is breached
    print("Overall margin limit breached")

    # ...cancel all SELL open orders, if it is not empty
    if not df_openord.empty:
        m_opord = (df_openord.action == 'SELL')
        df_sells = df_openord[m_opord]

        # cancel the SELL openorders
        cancel_list = [o for o in ib.openOrders() if o.orderId in list(df_sells.orderId)]
        cancelled = [ib.cancelOrder(c) for c in cancel_list]

#         return None # abort the dynamic function. ***PLEASE UNCOMMENT THIS IN LIVE FUNCTION**** 

#... prepare the harvesting BUY orders for those without them

# get the positions
df_p = util.df(ib.positions())

# get the position contracts
df_pc = util.df(list(df_p.contract))
df_pc = df_pc[list(df_pc)[:6]]
df_pc = df_pc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

df_pc = df_pc.join(df_p[['position', 'avgCost']])

df_pc.head()

df_pc = df_pc.assign(avgCost=np.where(df_pc.secType == 'OPT', df_pc.avgCost, df_pc.avgCost))

# initialize buy orders to be equal to df_pc
df_buy = df_pc
df_buy = df_buy.assign(dte=[get_dte(d) for d in df_buy.expiration])

# get the latest prices
contracts = ib.qualifyContracts(*[Contract(conId=c) for c in df_buy.conId])
df_buy = df_buy.assign(contract=contracts)

tickers = ib.reqTickers(*contracts)
optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {symbol: undPrice}

# Put the option prices
df_buy = df_buy.assign(optPrice = df_buy.conId.map(optPrices))
df_buy = df_buy.assign(optPrice = df_buy.optPrice.fillna(prec)) # make NaN to lowest price (prec)

# get the harvest price
df_buy = df_buy.assign(hvstPrice=[get_prec(min(hvstPricePct(d)*c,mp*0.9),prec) 
                                  for d, c, mp in zip(df_buy.dte, df_buy.avgCost, df_buy.optPrice)])

#...prepare the SELL orders

# get remaining quantities
dfrq = remqty_nse(ib)
# update target with remaining quantities
dft = pd.read_pickle(fspath+'targets.pickle')

# prepare the target and refresh
dft = dft.assign(remqty=dft.symbol.map(dfrq.remqty.to_dict()))   # remaining quantities
dft = dft[dft.remqty > 0].reset_index(drop=True) # remove blacklisted (remqty <= 0)

dft = upd(ib, dft)

# adjust optprice for fall-rise risk
risky = riskyprice(dft, prec)
dft = dft.assign(expPrice = 
           dft.assign(risky=dft.optId.map(risky))[['expPrice', 'risky']].max(axis=1))

if not df_openord.empty:
#... As there are some open orders, 
#    ..prepare the remaining 'BUY' orders
#    ..cancel all the 'SELL' orders to cancel
#    ..update prices and roms for dft
#    ..prepare the new harvest 'SELL' orders

    # remove the exising buys
    df_bx = df_openord[df_openord.action == 'BUY']
    df_buy = df_buy[~df_buy.conId.isin(list(df_bx.conId))]
    
    fill_symbols = []
    if ib.fills():     # get fill symbols, if there are any SELL fills!
        df_fc = util.df(list(util.df(ib.fills())['contract'])) # filled contracts
        df_fc = df_fc[list(df_fc)[:6]]
        df_fc = df_fc.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})
        df_fc

        df_fe = util.df(list(util.df(ib.fills())['execution'])) # executed contracts
        df_fe = df_fe[list(df_fe)[4:13]].drop('liquidation', 1) 

        df_fill = df_fc.join(df_fe)

        # get the fill symbols
        fill_symbols = list(df_fill.symbol.unique())

        # identify the recalc orders which needs to be cancelled and reordered
        df_rcalc = dft[dft.symbol.isin(fill_symbols)]

        # cancel the SELL orders
        sell_ordId = df_openord[df_openord.action == 'SELL'].set_index('conId')['orderId'].to_dict()
        df_cancel_rc = df_rcalc.assign(ordId=df_rcalc.optId.astype('int32').map(sell_ordId)).dropna()
        cancelords = [o for o in ib.openOrders() if o.orderId in list(df_cancel_rc.ordId.astype(int))]
        rc_cancelled = [ib.cancelOrder(c) for c in cancelords]

        # jackup expPrice to cover the risk
        df_rcalc = df_rcalc.assign(expPrice=get_prec(df_rcalc.expPrice*jackup,0.05))
        # re-calculate expRom
        df_rcalc = df_rcalc.assign(expRom=df_rcalc.expPrice*df_rcalc.lot/df_rcalc.margin*365/df_rcalc.dte)
        
        # remove the recalc optIds from dft
        dft=dft[~dft.optId.isin(list(df_rcalc.optId))]

        # re-add df_rcalc to dft and pickle
        dft = grp_opts(pd.concat([dft, df_rcalc[dft.columns]]))

        # write to pickle
        dft.to_pickle((fspath+'targets.pickle'))
        
    else: # no fills but have open orders!
        m_opord = (df_openord.action == 'SELL')
        df_sells = df_openord[m_opord]

        # cancel the SELL openorders
        cancel_list = [o for o in ib.openOrders() if o.orderId in list(df_sells.orderId)]
        cancelled = [ib.cancelOrder(c) for c in cancel_list]
        
#...prepare to place the orders
buy_orders = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) 
                      for position, hvstPrice in zip(df_buy.position, df_buy.hvstPrice)]

hco = zip(contracts, buy_orders)
l_hco = list(hco)

sell_blks = trade_blocks(ib, dft, exchange=exchange)

# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# WARNING: THE FOLLOWING CODE PLACES TRADES
# _________________________________________
if l_hco:
    hvstTrades = [ib.placeOrder(c, o) for c, o in l_hco]
if sell_blks:
    sowTrades = doTrades(ib, sell_blks)

In [ ]:
dyn